## MILP Approach for Small Instances

fixed cost first echelon vehicles - 100  
fixed cost second echelon vehicles - 50

In [ ]:
# Dummy


# Assuming costs and decision variables are given in appropriate structures, such as dictionaries or arrays

# Sets
T = [...]  # List of first-echelon vehicles
V = [...]  # List of second-echelon vehicles
DU_S = [...]  # Union of Distribution Centers (D) and Satellites (S)
SU_CU_O = [...]  # Union of Satellites (S), Customers (C), and Collaboration Points (O)

# Parameters
C = {...}  # Cost matrix: C[i][j] is the cost from node 'i' to node 'j'
F_t = {...}  # Fixed costs for first-echelon vehicles
F_v = {...}  # Fixed costs for second-echelon vehicles

# Decision variables
R = {...}  # R[i][j][t] decision variable for first-echelon
X = {...}  # X[i][j][t] decision variable for second-echelon
U_t = {...}  # U_t[t] binary variable for first-echelon vehicles
U_v = {...}  # U_v[v] binary variable for second-echelon vehicles

# Objective function
objective = (
    sum(sum(C[i][j] * R[i][j][t] for j in DU_S) for i in DU_S for t in T) +
    sum(sum(sum(C[i][j] * X[i][j][v] for j in SU_CU_O) for i in SU_CU_O) for v in V) +
    sum(F_t[t] * U_t[t] for t in T) +
    sum(F_v[v] * U_v[v] for v in V)
)


In [ ]:
from gurobipy import Model, GRB, quicksum

# Create the model
model = Model("last_mile_distribution")

# Sets
T = [...]  # List of first-echelon vehicles
V = [...]  # List of second-echelon vehicles
DU_S = [...]  # Union of Distribution Centers (D) and Satellites (S)
SU_CU_O = [...]  # Union of Satellites (S), Customers (C), and Collaboration Points (O)
C = [...]  # Set of customers
S = [...]  # Set of satellites
D = [...]  # Set of distribution centers

# Parameters
C_ij = {...}  # Cost matrix: C_ij[i][j] is the cost from node 'i' to node 'j'
F_t = {...}  # Fixed costs for first-echelon vehicles
F_v = {...}  # Fixed costs for second-echelon vehicles
d_c = {...}  # Demand of each customer
K_1 = {...}  # Capacity of first-echelon vehicles
M = 10000  # A sufficiently large constant

# Decision variables
R_ij_t = model.addVars(DU_S, DU_S, T, vtype=GRB.CONTINUOUS, name="R_ij_t")
X_ij_v = model.addVars(SU_CU_O, SU_CU_O, V, vtype=GRB.CONTINUOUS, name="X_ij_v")
U_t = model.addVars(T, vtype=GRB.BINARY, name="U_t")
U_v = model.addVars(V, vtype=GRB.BINARY, name="U_v")
Y_c_t = model.addVars(C, T, vtype=GRB.BINARY, name="Y_c_t")
Q_s_c = model.addVars(S, C, vtype=GRB.BINARY, name="Q_s_c")

# Objective function
model.setObjective(
    quicksum(C_ij[i][j] * R_ij_t[i, j, t] for i in DU_S for j in DU_S for t in T) +
    quicksum(C_ij[i][j] * X_ij_v[i, j, v] for i in SU_CU_O for j in SU_CU_O for v in V) +
    quicksum(F_t[t] * U_t[t] for t in T) +
    quicksum(F_v[v] * U_v[v] for v in V),
    GRB.MINIMIZE
)

# Constraints

# (ii) Flow conservation constraint
for j in DU_S:
    for t in T:
        model.addConstr(quicksum(R_ij_t[i, j, t] for i in DU_S) + quicksum(R_ij_t[j, i, t] for i in DU_S) == 0, name=f"FlowConservation_{j}_{t}")

# (iii) Vehicle usage constraint
for t in T:
    model.addConstr(quicksum(R_ij_t[i, j, t] for i in D for j in S) <= U_t[t], name=f"VehicleUsage_{t}")

# (iv) Service coverage constraint
for c in C:
    for t in T:
        model.addConstr(quicksum(R_ij_t[p_c, s, t] for s in S) >= Y_c_t[c, t], name=f"ServiceCoverage_{c}_{t}")

# (v) Demand satisfaction constraint
for t in T:
    model.addConstr(quicksum(d_c[c] * Y_c_t[c, t] for c in C) <= K_1[t] * U_t[t], name=f"DemandSatisfaction_{t}")

# (vi) Customer assignment constraint
for c in C:
    model.addConstr(quicksum(Y_c_t[c, t] for t in T) == 1, name=f"CustomerAssignment_{c}")

# (vii) Sufficient service constraint
for c in C:
    for s in S:
        for t in T:
            model.addConstr(M * (2 - Y_c_t[c, t] - Q_s_c[s, c]) + quicksum(R_ij_t[k, s, t] for k in DU_S) >= 1, name=f"SufficientService_{c}_{s}_{t}")

# (viii) Arc-flow restriction
for i in D + S:
    for j in D + S:
        if i != j:
            model.addConstr(R_ij_t[i, j] == 0, name=f"ArcFlowRestriction_{i}_{j}")

# (ix) Satellite assignment constraint
for c in C:
    for m in L:
        model.addConstr(quicksum(Q_s_c[s, c] for s in S if c in C_m) == 1, name=f"SatelliteAssignment_{c}_{m}")




In [ ]:
# Optimize the model
model.optimize()

# Output the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    for var in model.getVars():
        print(f"{var.varName}: {var.x}")
else:
    print(f"Optimization ended with status {model.status}.")